##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 畳み込みニューラルネットワークで、より洗練された画像を使用する

前の授業では、CNN（畳み込みニューラルネットワーク）を使用して手書きの数字をより効率的に認識する方法を見ました。この授業では次のレベルへ進んで、猫と犬の実際の画像を認識し、入力画像をどちらかに分類します。特に、手書き認識では、すべての画像を同じサイズと形状に揃えることで作業が少し楽になりましたし、すべてモノクロでした。現実世界の画像は、そうは行きません。形状もアスペクト比もさまざまですし、通常はカラーです。

そこで、タスクの一部として、データを処理する必要があります。特に、サイズ変更して、形状を揃える必要があります。 

以下のステップに従います。

1.   猫と犬のサンプルデータを調べる
2.   ２つの違いを認識するニューラルネットワークを構築し、訓練する
3.   訓練と検証の精度を評価する

## サンプルデータを調べる

まず、サンプルデータをダウンロードしましょう。猫と犬の2,000枚のJPG写真があるzipファイルです。これをローカルの`/tmp`に解凍します。

**注：**この練習問題で使用される2,000枚の画像は、Kaggleにある25,000枚の画像を含む[「犬と猫」データセット](https://www.kaggle.com/c/dogs-vs-cats/data)データセットから抜粋されたものです。ここでは、学習目的に合わせて、訓練時間を短縮するためにデータセット全体のうちの一部を使用します。

In [ ]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O /tmp/cats_and_dogs_filtered.zip

次のPythonコードは、オペレーティングシステムライブラリを使用してファイルシステムへのアクセスできるようにし、zipfileライブラリを使用してデータを解凍できるようにします。 

In [ ]:
import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

zipファイルの内容はベースディレクトリの`/tmp/cats_and_dogs_filtered`に解凍されます。このディレクトリには、訓練用データセットと検証用データセットが入っている`train`サブディレクトリと`validation`サブディレクトリがあります。（訓練、検証、およびテストセットの復習については、「機械学習入門講座」を参照してください。）そして、それぞれに`cats`サブディレクトリと`dogs`サブディレクトリがあります。

要約すると、訓練セットは、「猫とはこういうものだ」、「犬とはこういうものだ」などをニューラルネットワークモデルに伝えるために使用されるデータです。検証データセットは、ニューラルネットワークが訓練の一環として見ていない猫と犬の画像であり、画像が猫か犬かの評価をどのくらい良くできるかをテストできます。

このサンプルで注意すべきことは、画像に猫または犬のラベルを明示的には付けていないことです。以前の手書きの例を思い出してみると、「これは1」、「これは7」などのラベルを付けました。後で、画像ジェネレーターと呼ばれるものが使用されるのを見ていきますが、これはサブディレクトリから画像を読み取り、そのサブディレクトリの名前から自動的にラベルを付けるようコード化されています。たとえば、「training」ディレクトリに「cats」ディレクトリと「dogs」ディレクトリがあるとします。画像ジェネレーターが自動的に適切なラベルを画像につけるため、コーディングの手間が省けます。 

これらの各ディレクトリを定義しましょう。

In [ ]:
base_dir = '/tmp/cats_and_dogs_filtered'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# 訓練用の猫/犬の写真があるディレクトリ
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

# 検証用の猫/犬の写真があるディレクトリ
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')


では、`cats`と`dogs`の`train`ディレクトリ内のファイル名を見てみましょう。（ファイル名の規則は、`validation`ディレクトリでも同じです）

In [ ]:
train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

print(train_cat_fnames[:10])
print(train_dog_fnames[:10])

`train`ディレクトリと`validation`ディレクトリ内の猫の画像と犬の画像の合計数を調べてみましょう。

In [ ]:
print('total training cat images :', len(os.listdir(train_cats_dir)))
print('total training dog images :', len(os.listdir(train_dogs_dir)))

print('total validation cat images :', len(os.listdir(validation_cats_dir)))
print('total validation dog images :', len(os.listdir(validation_dogs_dir)))

猫と犬の両方とも、訓練用画像は1,000枚、検証用画像は500枚あります。

今度は、数枚の写真を見て、猫と犬のデータセットがどう見えるか理解しましょう。まず、matplotパラメータを構成します。

In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# グラフのパラメータ。画像を4x4の構成で出力します。
nrows = 4
ncols = 4

pic_index = 0 # 画像を反復処理するためのインデックス

では、８枚の猫の写真と８枚の犬の写真のバッチを表示します。セルを再実行して、毎回、新しいバッチを見ることができます。


In [ ]:
# matplotlibの図をセットアップして、4x4の画像が収まるサイズにします
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=8

next_cat_pix = [os.path.join(train_cats_dir, fname) 
                for fname in train_cat_fnames[pic_index-8:pic_index] 
               ]

next_dog_pix = [os.path.join(train_dogs_dir, fname) 
                for fname in train_dog_fnames[pic_index-8:pic_index]
               ]

for i, img_path in enumerate(next_cat_pix+next_dog_pix):
  # サブプロットをセットアップします。サブプロットインデックスは1から始まります。
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # 軸（グリッド線）は表示しません

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()


この行列（グリッド）の画像を見てもよくわからないかもしれませんが、ここでの重要な注意として、前の授業との大きな違いは、これらの画像は形状とサイズがさまざまなことです。手書き認識の例では、28x28のグレースケール画像を使用しました。今回はカラーで、形状もさまざまです。これらを使用してニューラルネットワークを訓練する前に、画像に手を加える必要があります。これは次のセクションで見ます。


データがどのようなものか理解できたところで、次のステップは、これらの画像から猫か犬かを認識するように訓練されるモデルを定義することです。 


## 小さなモデルを初めから構築して、72%以上の精度を得る


前のセクションでは、画像の形状とサイズがさまざまであることを確認しました。それらを処理するようにニューラルネットワークを訓練するには、同じサイズにする必要があります。このために150x150を選び、画像をこの形状に前処理するコードを簡単に見ていきます。

しかし、続ける前に、モデルの定義を開始しましょう。

まず初めに、tensorflowをインポートします。

In [ ]:
import tensorflow as tf

次に、以前と同様にSequential層を定義し、最初にいくつかの畳み込み層を追加します。今回は入力形状パラメータに注目してください。前の例では28x28x1でした。画像が28x28のグレースケールだったからです（色深度は８ビット、つまり1バイト）。今回はサイズは150x150、色深度は3（24ビット、つまり3バイト）です。 


次に、前の例と同様に２つの畳み込み層を追加し、最終結果をフラット化して、密結合された層に与えます。


最後に、密結合層を追加します。 

２クラスの分類問題、つまり、*２値分類問題* に直面しているため、ネットワークは [*sigmoid* 活性化関数](https://wikipedia.org/wiki/Sigmoid_function)で終了することに注意してください。これにより、ネットワークの出力は、０と１の間の単一のスカラになり、現在の画像がクラス１（クラス０ではなく）である確率をエンコードします。

In [ ]:
model = tf.keras.models.Sequential([
    # 入力形状が、150x150、3バイトの色という望ましい画像サイズであることに注目してください
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # 結果をフラット化して、DNNに与えます
    tf.keras.layers.Flatten(), 
    # 512ニューロンの隠れ層
    tf.keras.layers.Dense(512, activation='relu'), 
    # 出力ニューロンは１つのみ。結果は０から1までの値を含み、０は一方のクラス（cats）を表し、１は他方のクラス（dogs）を表します。
    tf.keras.layers.Dense(1, activation='sigmoid')  
])

model.summary()メソッドコールは、NN（ニューラルネットワーク）のサマリーを表示します。 

In [ ]:
model.summary()

"output shape" 列は、特徴マップのサイズが、連続する各層でどのように増加するかを示します。畳み込み層は、パディングにより少しだけ、特徴マップのサイズを小さくし、各プーリング層は長さを半分にします。

次に、モデルの訓練の仕様を構成します。`binary_crossentropy`損失関数を使用してモデルを訓練します。これは２値分類問題であり、最終的なアクティベーションはシグモイドだからです。（損失評価の復習については、[機械学習入門講座](https://developers.google.com/machine-learning/crash-course/descending-into-ml/video-lecture)を参照してください。）`rmsprop`オプティマイザを`0.001`の学習率で使用します。訓練時、分類の精度を監視したいと思います。

**注**: この例では、[RMSprop最適化アルゴリズム](https://wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp)の使用が[確率的勾配降下法](https://developers.google.com/machine-learning/glossary/#SGD)（SGD）より望ましいです。RMSpropは学習率の調整を自動化するからです。（[Adam](https://wikipedia.org/wiki/Stochastic_gradient_descent#Adam)や[Adagrad](https://developers.google.com/machine-learning/glossary/#AdaGrad)など、他の最適化アルゴリズムも訓練時の学習率を自動的に調整し、ここでも同様に良く機能します。）

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

### データの前処理

ソースフォルダの写真を読み取って、`float32`テンソルに変換し、それらを（ラベルとともに）ネットワークに与えるデータジェネレーターをセットアップしましょう。訓練用画像のジェネレーターが１つ、検証用画像のジェネレーターが１つあります。ジェネレーターはサイズ150x150の20枚の画像とそれらのラベル（２値）のバッチを生成します。

すでにご存じかもしれませんが、ニューラルネットワークに入力されるデータは、ネットワークによる処理が容易になるように、通常、何らかの方法で正規化する必要があります。（生のピクセルを畳み込みニューラルネットワークに与えるのは一般的ではありません）この例では、画像の前処理として、ピクセル値を`[0, 1]`の範囲に正規化します。（もともと、すべての値は`[0, 255]`の範囲内です）

Kerasでは、これは`keras.preprocessing.image.ImageDataGenerator`クラスにより`rescale`パラメータを使用して行うことができます。`ImageDataGenerator`クラスを使用すると、`.flow(data, labels)`または`.flow_from_directory(directory)`を介して、拡張された画像バッチ（およびそのラベル）のジェネレーターをインスタンス化できます。これらのジェネレーターは、入力としてデータジェネレーターを受け入れるKerasモデルメソッド、すなわち、`fit`、`evaluate_generator`、および`predict_generator`で使用できます。

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# すべての画像は1./255で再スケールされます
train_datagen = ImageDataGenerator(rescale = 1.0/255.)
test_datagen  = ImageDataGenerator(rescale = 1.0/255.)

# --------------------
# train_datagenジェネレーターを使用した20のバッチで訓練用画像を流します
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))     
# --------------------
# test_datagenジェネレーターを使用した20のバッチで検証用画像を流します
# --------------------
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode  = 'binary',
                                                         target_size = (150, 150))


### 訓練
使用可能な2,000枚の画像すべてで15エポックにわたって訓練し、1,000枚のテスト画像すべてで検証しましょう。（実行には数分かかることがあります。）

エポックごとの値に注目してください。

エポックごとに４つの値があります。Loss（損失）、Accuracy（精度）、Validation Loss（検証の損失）、およびValidation Accuracy（検証の精度）です。 

「Loss（損失）」と「Accuracy（精度）」は、訓練の進捗状況を示す優れた指標です。訓練データの分類に関する推測を行い、それを既知のラベルと照合して評価し、結果を計算します。「Accuracy（精度）」は、正しい推測の割合です。検証の精度は、訓練で使用されなかったデータでの測定です。予測通り、これは少し低くなります。これが起こる理由については、この講座の過学習のセクションで学びます。


In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    steps_per_epoch=100,
    epochs=15,
    validation_steps=50,
    verbose=2)

###モデルの実行

では、モデルを使用して実際に実行される予測を見てみましょう。このコードでは、ファイルシステムから１つ以上のファイルを選択して、アップロードし、それらでモデルを実行して、オブジェクトが犬か猫かを推測できます。

In [ ]:
import numpy as np

from google.colab import files
from tensorflow.keras.preprocessing.image import img_to_array, load_img

uploaded=files.upload()

for fn in uploaded.keys():
 
  # 画像の推測
  path='/content/' + fn
  img = load_img(path, target_size=(150, 150))
  
  x = img_to_array(img)
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])
  
  classes = model.predict(images, batch_size=10)
  
  print(classes[0])
  
  if classes[0]>0:
    print(fn + " is a dog")
    
  else:
    print(fn + " is a cat")
 

### 中間表現の可視化

畳み込みニューラルネットワークが、どんな種類の特徴を学習するかを理解する楽しみの１つは、入力が畳み込みニューラルネットワークを通過する際にどのように変形されるか可視化することです。

訓練セットからランダムに猫または犬の画像を選んで、図を生成します。各行が層の出力であり、行内の各画像は、出力された特徴マップ内の特定のフィルタです。このセルを再実行して、さまざまな訓練用画像の中間表現を生成します。


In [ ]:
import numpy as np
import random

# 入力として画像を受け取り、
# 最初のモデルの後に前のモデルのすべての層の中間表現を出力する
# 新しいモデルを定義しましょう
successive_outputs = [layer.output for layer in model.layers[1:]]

#visualization_model = Model(img_input, successive_outputs)
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

# 訓練セットの猫または犬のランダムな入力画像を用意しましょう。
cat_img_files = [os.path.join(train_cats_dir, f) for f in train_cat_fnames]
dog_img_files = [os.path.join(train_dogs_dir, f) for f in train_dog_fnames]

img_path = random.choice(cat_img_files + dog_img_files)
img = load_img(img_path, target_size=(150, 150))  # これはPIL画像です

x   = img_to_array(img)                           # 形状(150, 150, 3)のNumpy配列
x   = x.reshape((1,) + x.shape)                   # 形状(1, 150, 150, 3)のNumpy配列

# 1/255で再スケールします
x /= 255.0

# 画像をネットワークに通して、
# この画像のすべての中間表現を取得しましょう。
successive_feature_maps = visualization_model.predict(x)

# これらは層の名前なので、プロットの一部として使用できます
layer_names = [layer.name for layer in model.layers]

# -----------------------------------------------------------------------
# 表現を表示しましょう
# -----------------------------------------------------------------------
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  
  if len(feature_map.shape) == 4:
    
    #-------------------------------------------
    # これは畳み込み層とmaxpool層についてのみ行い、全結合層については行いません
    #-------------------------------------------
    n_features = feature_map.shape[-1]  # 特徴マップ内の特徴の数
    size       = feature_map.shape[ 1]  # 特徴マップの形状(1, size, size, n_features)
    
    # 画像をこのマトリックスに並べます
    display_grid = np.zeros((size, size * n_features))
    
    #-------------------------------------------------
    # 特徴を後処理して、見やすくします
    #-------------------------------------------------
    for i in range(n_features):
      x  = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std ()
      x *=  64
      x += 128
      x  = np.clip(x, 0, 255).astype('uint8')
      display_grid[:, i * size : (i + 1) * size] = x # 各フィルタを水平行列に並べます

    #-----------------
    # 行列を表示します
    #-----------------

    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis') 

ご覧のように、画像の生のピクセルから、徐々に抽象的でコンパクトな表現になっていきます。表現のダウンストリームは、ネットワークが注意を払うものを強調し始め、「アクティベート」される特徴が徐々に少なくなり、ほとんどがゼロにセットされます。これを「スパース性」といいます。表現のスパース性は、ディープラーニングの重要な特徴です。


これらの表現は、画像の元のピクセルに関して、ますます少ない情報しか伝えませんが、画像のクラスに関しては、より洗練された情報を伝えます。畳み込みニューラルネットワーク（または一般に、深層ネットワーク）は、情報を蒸留するパイプラインと考えることができます。

### モデルの精度と損失を評価する

訓練時に収集された訓練と検証に関する精度および損失をプロットしましょう。

In [ ]:
#-----------------------------------------------------------
# 各訓練エポックについて、訓練およびテストデータセットの
# 結果リストを取得します
#-----------------------------------------------------------
acc      = history.history['accuracy']
val_acc  = history.history['val_accuracy']
loss     = history.history['loss']
val_loss = history.history['val_loss']

epochs   = range(len(acc)) # エポックの数を取得する

#------------------------------------------------
# エポックごとの訓練および検証の精度をプロットします
#------------------------------------------------
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# エポックごとの訓練および検証の損失をプロットします
#------------------------------------------------
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')

ご覧のように、ファッションアイテム分類で発生したのと同様に**過学習**になっています。訓練の精度（青）は100%に近づきますが、検証の精度（緑）は70%まで落ちています。検証の損失は、わずか5エポック後に最小値に達しています。

訓練サンプルが比較的少ないため（2,000）、過学習が一番の懸念です。過学習は、モデルが触れたサンプルが少なく、新しいデータに一般化できないパターンを学習した場合、つまり、モデルが推測を行うのに無関係な特徴を使用し始めたときに発生します。たとえば、人間として、木こりの画像３つと、船乗りの画像３つしか表示されておらず、その中で帽子をかぶっているのが木こりだけだった場合、帽子をかぶることが船乗りではなく木こりである印と考え始めるかもしれません。その場合、かなりお粗末な木こりと船乗りの分類器を作ることになります。

過学習は、機械学習における中心的問題です。モデルのパラメータを特定のデータセットに合わせる場合、モデルが学習した表現が、以前に見たことがないデータに適用可能かどうか、どうしたら確認できるでしょうか。訓練データに固有のものを学習しないようにするには、どうしたらよいでしょうか？


次の練習問題では、猫対犬の分類モデルで過学習を防ぐ方法を見ていきます。

## クリーンアップ

次の練習問題を行う前に、次のセルを実行してカーネルを終了し、メモリリソースを解放してください。

In [ ]:
import os, signal

os.kill(os.getpid(), signal.SIGKILL)